# 작업형 1

#### 01. mtcars2 데이터셋을 불러와 mpg 컬럼의 상위 10번째 값으로 상위 10개 값을 변환한 후 drat가 4 이상인 값에 대해 mpg의 평균을 구하여라.

In [36]:
import pandas as pd

df = pd.read_csv('./data/기출복원/02회/mtcars2.csv')

df2 = df[['mpg','drat']].copy()
df2_rank = df2.sort_values('mpg', ascending=False).reset_index()[['mpg','drat']]

mpg_val = df2_rank.loc[9]['mpg']

df2_rank['mpg'][:9] = mpg_val

# print(df2_rank)
# print(mpg_val)

result = df2_rank[df2_rank['drat'] >= 4]['mpg'].mean()
print(result)

20.671428571428574


#### 2. 앞의 데이터셋을 새로 불러와 첫 번째 행부터 순서대로 80%까지의 데이터를 훈련 데이터로 추출한 후 disp 컬럼의 결측값을 disp 컬럼의 중앙값으로 대체하고 대체 전 후의 disp 변수의 표준편차 값의 차이를 구하여라. (단, 차이는 빼는 순서와 관계없이 절댓값을 취하여 표시하고 소수점은 넷째 자리에서 반올림할 것)

In [65]:
import pandas as pd

df = pd.read_csv('./data/기출복원/02회/mtcars2.csv')
# print(df.shape)

portion = int(df.shape[0]*0.8)
df2 = df[:portion]

disp_med = df2['disp'].median()  # 중앙값 구하기

before = df2['disp'].copy()  # 이전 데이터
after = before.copy()

after[df2['disp'].isna() == True] = disp_med  # 이후 데이터
# print(before.isna().sum())
# print(after.isna().sum())

std_bef = before.std()
std_aft = after.std()

result = round(abs(std_aft - std_bef), 3)
print(result)

7.966


#### 3. gehan 데이터 셋을 불러와 time 컬럼에서 이상값의 합을 구하여라.(단, 이상값은 평균에서 1.5 표준편차 이상으로 벗어난 값으로 정의함)

In [61]:
import pandas as pd

df = pd.read_csv('./data/기출복원/02회/gehan.csv')

# print(df.head())
# print(df.shape)

time = df['time'].copy()
std_time = time.std()
mean_time = time.mean()

condition = (time >= mean_time+1.5*std_time) | (time <= mean_time-1.5*std_time)
a = time[condition]
result = a.sum()

print(result)

133


---

# 작업형2

#### 1. 아래는 뇌졸중(stroke)에 대한 환자들의 임상적 변수에 관련한 데이터의 일부이다. 주어진 데이터를 이용하여 예측 모형을 만들고 아래에 따라 CSV 파일을 생성하시오. (단, 제출 전 두 개이상의 모형의 성능을 비교하여 가장 우수한 모형을 선정할 것)

In [133]:
### 데이터 불러오기
import pandas as pd

X_train = pd.read_csv('data/기출복원/02회/stroke_X_train.csv')  # 4087
y_train = pd.read_csv('data/기출복원/02회/stroke_y_train.csv')
X_test = pd.read_csv('data/기출복원/02회/stroke_X_test.csv')  # 1022

### EDA

    # 범주 'gender','ever_married','work_type','Residence_type','smoking_status'
    # 수치 'age','hypertension','heart_disease','avg_glucose_level','bmi'
    
# print(X_train.isna().sum(), '\n\n')
# print(X_test.isna().sum())
    # 결측치 'bmi' train(165) test(36)

# 'id' -> 불필요. 제거
X_train = X_train.drop(columns = 'id').copy()
y_train = y_train.drop(columns = 'id').copy()
ID = X_test.pop('id')

# 'gender' -> 원-핫 인코딩
# 'age' float -> int
X_train['age'] = X_train['age'].astype('int64')
X_test['age'] = X_test['age'].astype('int64')

# 'hypertension' -> train - 0(3684) 1(403) / test - 0(927) 1(95) -> 인코딩 완료(범주)
# 'heart_disease' -> train - 0(3863) 1(224) / test - 0(970) 1(52) -> 인코딩 완료(범주)
# 'ever_married' -> 'Yes','No' -> 원-핫 인코딩
# 'work_type' -> 'Private','Self-employed','children','Govt_job','Never_worked' -> 원-핫 인코딩
# 'Residence_type' -> 'Urban','Rural' -> 원-핫 인코딩
# 'smoking_status' -> 'never smoked','Unknown','formerly smoked','smokes' -> 'Unknown' 결측치로 판단
    # 'Unknown'(1240) 너무 많음. 일부 제거할 시 데이터 손실 심각 -> 변수 제거
X_train = X_train.drop(columns = 'smoking_status')
X_test = X_test.drop(columns = 'smoking_status')

# 'avg_glucose_level' -> mean(106.2) std(45.7) / max(271.7) min(55.2)
# 'bmi' -> 결측치 처리
train_bmi = X_train['bmi'].mean()
X_train['bmi'] = X_train['bmi'].fillna(train_bmi).copy()

test_bmi = X_test['bmi'].mean()
X_test['bmi'] = X_test['bmi'].fillna(test_bmi).copy()

### 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=2023)

### 인코딩
from sklearn.preprocessing import OneHotEncoder

X_train_obj = X_train.select_dtypes('object').copy()
X_val_obj = X_val.select_dtypes('object').copy()
X_test_obj = X_test.select_dtypes('object').copy()

oh = OneHotEncoder(sparse=False).fit(X_train_obj)

X_train_oh = oh.transform(X_train_obj)
X_val_oh = oh.transform(X_val_obj)
X_test_oh = oh.transform(X_test_obj)

### 스케일링
from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(exclude = 'object').copy()
X_val_num = X_val.select_dtypes(exclude = 'object').copy()
X_test_num = X_test.select_dtypes(exclude = 'object').copy()

scale = StandardScaler().fit(X_train_num)

X_train_scale = scale.transform(X_train_num)
X_val_scale = scale.transform(X_val_num)
X_test_scale = scale.transform(X_test_num)

In [14]:
### 1. 데이터 불러오기
import pandas as pd

X_train = pd.read_csv('data/기출복원/02회/stroke_X_train.csv')
y_train = pd.read_csv('data/기출복원/02회/stroke_y_train.csv')
X_test = pd.read_csv('data/기출복원/02회/stroke_X_test.csv')


### 2. EDA - head(), info(), describe(), isnull().sum(), value_counts()

        # 'id' 불필요
        # 'age' 변수 조절
        # 범주형 ['gender','ever_married','work_type','Residence_type','smoking_status']
                #    2           2             5             2                4
        # 'bmi' 결측치
        # 'smoking_status'에서 'Unknown'은 결측치 아닌 결측치. 너무 많음 -> 'smoking_status' 제거

### 3. 데이터 전처리
# 1) 불필요 변수 제거
X_train = X_train.drop('id', axis = 1)
y_train = y_train.drop('id', axis = 1)
test_id = X_test.pop('id')

# 2) 결측치 처리
mean_bmi = X_train['bmi'].mean()

X_train['bmi'] = X_train['bmi'].fillna(mean_bmi)
X_test['bmi'] = X_test['bmi'].fillna(mean_bmi)

X_train = X_train.drop('smoking_status', axis = 1)
X_test = X_test.drop('smoking_status', axis = 1)

# 3) 범주형 변수 처리

# 4) 수치형 변수 처리
X_train['teen'] = pd.cut(X_train['age'], bins = list(range(0,100,10))).astype('object')
X_test['teen'] = pd.cut(X_test['age'], bins = list(range(0,100,10))).astype('object')

X_train = X_train.drop('age', axis = 1)
X_test = X_test.drop('age', axis = 1)

# 5) 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size = 0.3, 
                                                  random_state = 2022, 
                                                  stratify = y_train)


# 6) 인코딩
X_train_obj = X_train.select_dtypes('object').copy()
X_val_obj = X_val.select_dtypes('object').copy()
X_test_obj = X_test.select_dtypes('object').copy()

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse = False).fit(X_train_obj)

X_train_oh = ohe.transform(X_train_obj)
X_val_oh = ohe.transform(X_val_obj)
X_test_oh = ohe.transform(X_test_obj)

# 7) 스케일링
X_train_num = X_train.select_dtypes(exclude = 'object').copy()
X_val_num = X_val.select_dtypes(exclude = 'object').copy()
X_test_num = X_test.select_dtypes(exclude = 'object').copy()

from sklearn.preprocessing import StandardScaler

scale = StandardScaler().fit(X_train_num)

X_train_scale = scale.transform(X_train_num)
X_val_scale = scale.transform(X_val_num)
X_test_scale = scale.transform(X_test_num)

# 8) 데이터셋 준비
import numpy as np

X_train = np.concatenate([X_train_oh, X_train_scale], axis = 1)
X_val = np.concatenate([X_val_oh, X_val_scale], axis = 1)
X_test = np.concatenate([X_test_oh, X_test_scale], axis = 1)

y_train = y_train.values.flatten()
y_val = y_val.values.flatten()


### 4. 모델 학습 및 예측
# 모델 학습
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

rf = RandomForestClassifier()
model_rf = rf.fit(X_train, y_train)

xgb = XGBClassifier()
model_xgb = xgb.fit(X_train, y_train)

lgb = LGBMClassifier()
model_lgb = lgb.fit(X_train, y_train)

# 모델 검증
from sklearn.metrics import accuracy_score

pred_rf = model_rf.predict(X_val)
pred_xgb = model_xgb.predict(X_val)
pred_lgb = model_lgb.predict(X_val)

acc_rf = accuracy_score(y_val, pred_rf)
acc_xgb = accuracy_score(y_val, pred_xgb)
acc_lgb = accuracy_score(y_val, pred_lgb)

# 예측
y_pred = model_xgb.predict(X_test)

### 5. 결과제출
obj = {'id':test_id,
       'stroke':y_pred}
result = pd.DataFrame(obj)

result.to_csv('12345.csv', index = False)

C:\Users\idjs1\.conda\envs\big_ana\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:24:52] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
